In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [5]:
driver = webdriver.Chrome()

In [6]:
driver.get('https://www.transfermarkt.com/wettbewerbe/europa')

In [7]:
driver.maximize_window() # zwiekszamy wielkosc okna
time.sleep(1)

In [8]:
wait = WebDriverWait(driver,10)

In [9]:
wait.until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,'//iframe[@title="SP Consent Message"]')))
wait.until(EC.element_to_be_clickable((By.XPATH,"//button[contains(@title,'ACCEPT ALL')]"))).click()

In [10]:
country_map = wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="europa_Map"]')))
countries = country_map.find_elements_by_tag_name('area')
[country.get_attribute('title') for country in countries]

['Bosnia & Herzegovina',
 'Croatia',
 'Serbia',
 'Slovenia',
 'Hungary',
 'Belgium',
 'Luxembourg',
 'Switzerland',
 'Austria',
 'Czech Republic',
 'Poland',
 'France',
 'Netherlands',
 'Romania',
 'Germany',
 'Portugal',
 'Spain',
 'Italy',
 'Greece',
 'Israel',
 'Bulgaria',
 'Ukraine',
 'Türkiye',
 'Russia',
 'Denmark',
 'Sweden',
 'Northern Ireland',
 'Ireland',
 'England',
 'Scotland',
 'Iceland',
 'Finland',
 'Norway',
 'Albania',
 'Cyprus',
 'Estonia',
 'Belarus',
 'Slovakia',
 'Latvia',
 'Lithuania',
 'Wales',
 'Georgia',
 'Faroe Islands',
 'Malta',
 'Armenia',
 'Azerbaijan',
 'Moldova',
 'Kazakhstan',
 'Macedonia',
 'Kosovo',
 'Montenegro',
 'Gibraltar',
 'Liechtenstein',
 'San Marino',
 'Andorra']

In [11]:
country_dict = {country.get_attribute('title'): country.get_attribute('href') for country in countries}

In [12]:
country_driver = webdriver.Chrome()
country_driver.get(country_dict['England'])

In [13]:
country_wait = WebDriverWait(country_driver,10)

In [14]:
country_wait.until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,'//iframe[@title="SP Consent Message"]')))
country_wait.until(EC.element_to_be_clickable((By.XPATH,"//button[contains(@title,'ACCEPT ALL')]"))).click()

In [15]:
league = country_wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="yw1"]/table/tbody/tr[2]/td[1]/table/tbody/tr/td[2]/a')))
country_driver.get(league.get_attribute('href'))

In [16]:
teams = country_driver.find_elements_by_xpath('//*[@id="yw1"]/table/tbody/tr')
len(teams)

20

In [17]:
[team.text for team in teams]

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [18]:
teams_link = [team.find_elements_by_tag_name('td')[2].find_elements_by_tag_name('a')[0].get_attribute('href') for team in teams]
teams_link

['https://www.transfermarkt.com/manchester-city/kader/verein/281/saison_id/2021',
 'https://www.transfermarkt.com/liverpool-fc/kader/verein/31/saison_id/2021',
 'https://www.transfermarkt.com/chelsea-fc/kader/verein/631/saison_id/2021',
 'https://www.transfermarkt.com/manchester-united/kader/verein/985/saison_id/2021',
 'https://www.transfermarkt.com/tottenham-hotspur/kader/verein/148/saison_id/2021',
 'https://www.transfermarkt.com/arsenal-fc/kader/verein/11/saison_id/2021',
 'https://www.transfermarkt.com/leicester-city/kader/verein/1003/saison_id/2021',
 'https://www.transfermarkt.com/aston-villa/kader/verein/405/saison_id/2021',
 'https://www.transfermarkt.com/everton-fc/kader/verein/29/saison_id/2021',
 'https://www.transfermarkt.com/west-ham-united/kader/verein/379/saison_id/2021',
 'https://www.transfermarkt.com/wolverhampton-wanderers/kader/verein/543/saison_id/2021',
 'https://www.transfermarkt.com/newcastle-united/kader/verein/762/saison_id/2021',
 'https://www.transfermarkt.

In [19]:
country_driver.get(teams_link[0])

In [20]:
players = country_driver.find_elements_by_xpath('//*[@id="yw1"]/table/tbody/tr')
len(players)

22

In [21]:
players_link = [player.find_element_by_class_name('hauptlink').find_elements_by_tag_name('a')[0].get_attribute('href') for player in players]
players_link

['https://www.transfermarkt.com/ederson/profil/spieler/238223',
 'https://www.transfermarkt.com/zack-steffen/profil/spieler/221624',
 'https://www.transfermarkt.com/scott-carson/profil/spieler/14555',
 'https://www.transfermarkt.com/ruben-dias/profil/spieler/258004',
 'https://www.transfermarkt.com/aymeric-laporte/profil/spieler/176553',
 'https://www.transfermarkt.com/john-stones/profil/spieler/186590',
 'https://www.transfermarkt.com/nathan-ake/profil/spieler/177476',
 'https://www.transfermarkt.com/oleksandr-zinchenko/profil/spieler/203853',
 'https://www.transfermarkt.com/benjamin-mendy/profil/spieler/157495',
 'https://www.transfermarkt.com/joao-cancelo/profil/spieler/182712',
 'https://www.transfermarkt.com/kyle-walker/profil/spieler/95424',
 'https://www.transfermarkt.com/rodri/profil/spieler/357565',
 'https://www.transfermarkt.com/fernandinho/profil/spieler/26267',
 'https://www.transfermarkt.com/phil-foden/profil/spieler/406635',
 'https://www.transfermarkt.com/ilkay-gundogan

In [21]:
country_driver.get(players_link[0])

In [55]:
player_data = country_driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div')
player_data.text

'Full name:\nEderson Santana de Moraes\nDate of birth:\nAug 17, 1993\nPlace of birth:\nOsasco (SP)  \nAge:\n28\nHeight:\n1,88 m\nCitizenship:\n  Brazil\n  Portugal\nPosition:\nGoalkeeper\nFoot:\nleft\nPlayer agent:\nGestifute\nCurrent club:\nManchester City\nJoined:\nJul 1, 2017\nContract expires:\nJun 30, 2026\nDate of last contract extension:\nSep 1, 2021\nOutfitter:\nPuma\nSocial-Media:'

In [56]:
full_stats = country_driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[6]/a').get_attribute('href')
country_driver.get(full_stats)

In [57]:
detailed = country_driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div[1]/a[2]').get_attribute('href')
country_driver.get(detailed.replace('/2021/', '/2020/').replace('leistungsdaten', 'leistungsdatendetails'))
detailed

'https://www.transfermarkt.com/ederson/leistungsdaten/spieler/238223/saison/2021/plus/1#gesamt'

In [64]:
stats = country_driver.find_element_by_xpath('//*[@id="yw1"]/table')
stats.text

"competition\nclub\nSquad\n  PPG\n                    Total :   56 48 2.31 - - - - 3 - - 33 26 4.320'\nChampions League 12 12 2.58 - - - - - - - 5 7 1.080'\nPremier League 36 36 2.22 - - - - 3 - - 28 19 3.240'\nEFL Cup 3 - - - - - - - - - - - -\nFA Cup 5 - - - - - - - - - - - -"

In [65]:
stats = stats.text.split("\n")
total = stats[4].split(':   ')[1].split(' ')
total

['56', '48', '2.31', '-', '-', '-', '-', '3', '-', '-', '33', '26', "4.320'"]

In [66]:
if 'Champions League' in stats[5]:
    print(stats[5].split(' ')[3])

12


In [31]:
country_driver.back()

In [34]:
country_driver.back()

In [121]:
country_driver.forward()

In [35]:
market_value = country_driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[1]/div/div[2]/a').get_attribute('href')
country_driver.get(market_value)

In [142]:
point0 = country_driver.find_element_by_xpath('//*[@id="highcharts-0"]/div/span')
point0.text

'Dec 23, 2021\nMarket value: €50.00m\nclub: Manchester City\nAge : 28'

In [ ]:
from selenium.webdriver.common.action_chains import ActionChains

In [36]:
points = country_driver.find_elements_by_xpath('//*[@id="highcharts-0"]/*[name()="svg"]/*[name()="g"][5]/*[name()="g"][2]/*[name()="image"]')
len(points)

23

In [37]:
market_values = []
points.reverse()
for point in points:
    point.click()
    point0 = country_driver.find_element_by_xpath('//*[@id="highcharts-0"]/div/span')
    market_values.append(point0.text)
market_values

['Apr 3, 2013\nMarket value: €300Th.\nclub: Rio Ave FC\nAge : 19',
 'Jul 2, 2013\nMarket value: €300Th.\nclub: Rio Ave FC\nAge : 19',
 'Jul 25, 2014\nMarket value: €900Th.\nclub: Rio Ave FC\nAge : 20',
 'Feb 17, 2015\nMarket value: €600Th.\nclub: Rio Ave FC\nAge : 21',
 'Jul 1, 2015\nMarket value: €1.20m\nclub: SL Benfica\nAge : 21',
 'Feb 24, 2016\nMarket value: €1.20m\nclub: SL Benfica\nAge : 22',
 'Jul 29, 2016\nMarket value: €7.00m\nclub: SL Benfica\nAge : 22',
 'Feb 22, 2017\nMarket value: €12.00m\nclub: SL Benfica\nAge : 23',
 'Jun 26, 2017\nMarket value: €22.00m\nclub: SL Benfica\nAge : 23',
 'Oct 23, 2017\nMarket value: €25.00m\nclub: Manchester City\nAge : 24',
 'Jan 2, 2018\nMarket value: €35.00m\nclub: Manchester City\nAge : 24',
 'Mar 23, 2018\nMarket value: €45.00m\nclub: Manchester City\nAge : 24',
 'May 28, 2018\nMarket value: €50.00m\nclub: Manchester City\nAge : 24',
 'Oct 17, 2018\nMarket value: €60.00m\nclub: Manchester City\nAge : 25',
 'Dec 19, 2018\nMarket value: 

In [147]:
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
def get_driver():
    #caps = DesiredCapabilities().CHROME
    #caps["pageLoadStrategy"] = "normal"
    driver = webdriver.Chrome()#desired_capabilities=caps)
    return driver

In [171]:
import re
import ast
gk_stats = ['Squad', 'Appearances', 'PPG', 'Goals', 'Assists', 'Sub on', 'Sub off', 'Yellow cards', 'Seccond Yellow Cards', 'Red cards', 'Goals conceded', 'Clean sheets', 'Minutes played']
field_stats = ['Squad', 'Appearances', 'PPG', 'Goals', 'Assists', 'Own goals', 'Sub on', 'Sub off', 'Yellow cards', 'Seccond Yellow Cards', 'Red cards', 'Penalty goals', 'Minutes per goal', 'Minutes played']

def get_player_stats(player_url, name, driver):
    player = {}
    driver.get(player_url)
    player['Name'] = name
    player_data = driver.find_elements_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span')
    player_data = [data.text for data in player_data]
    for key, value in zip(player_data[::2], player_data[1::2]):
        player[key[:-1]] = value.strip()
    full_stats = driver.find_element_by_xpath("//*[contains(text(),'View full stats')]").get_attribute('href')
    market_value = driver.find_element_by_xpath("//*[contains(text(),'Market value details')]").get_attribute('href')
    driver.get(full_stats.replace('leistungsdaten', 'leistungsdatendetails') + "/saison/2020/plus/1")
    
    stats = driver.find_element_by_xpath('//*[@id="yw1"]/table')
    stats = stats.text.split("\n")
    total = stats[4].split(':   ')[1].split(' ')
    if 'Champions League' in stats[5]:
        player['CL Appearances'] = stats[5].split(' ')[3]
    else:
        player['CL Appearances'] = 0

    if player['Position'] == 'Goalkeeper':
        for key, value in zip(gk_stats, total):
            player[key] = value
    else:
        for key, value in zip(field_stats, total):
            player[key] = value

    driver.get(market_value)
    script = driver.find_element_by_xpath('/html/body/script[9]').get_attribute('innerHTML')
    series = re.search("'series':(.*),'credits'", script).group(1)
    market_values = ast.literal_eval(series)
    player['Market Values'] = market_values
    return player

In [172]:
def accept_cookies(driver):
    wait = WebDriverWait(driver,10)
    wait.until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,'//iframe[@title="SP Consent Message"]')))
    wait.until(EC.element_to_be_clickable((By.XPATH,"//button[contains(@title,'ACCEPT ALL')]"))).click()

In [200]:
import pandas as pd

def get_team(team_link, driver):
    driver.get(team_link)
    wait = WebDriverWait(driver,10)
    players = wait.until(lambda d: d.find_elements_by_xpath('//*[@id="yw1"]/table/tbody/tr'))
    player_links = [player.find_element_by_class_name('hauptlink').find_elements_by_tag_name('a')[0] for player in players]
    player_links = {player.text: player.get_attribute('href') for player in player_links}
    players = []
    team_name = team_link.split('/')[3]
    print(team_name)
    for name, player_link in list(player_links.items()):
        try:
            players.append(get_player_stats(player_link, name, driver))
        except:
            players.append({'Name': name})
    
    pd.DataFrame(players).to_csv(f"{team_name}.tsv", sep="\t", index=False)
    return players

def get_league(league_link, driver):
    players = []
    driver.get(league_link)
    accept_cookies(driver)
    teams = driver.find_elements_by_xpath('//*[@id="yw1"]/table/tbody/tr')
    team_links = [team.find_elements_by_tag_name('td')[2].find_elements_by_tag_name('a')[0].get_attribute('href') for team in teams]
    for team_link in team_links:
        try:
            players += get_team(team_link, driver)
        except:
            continue
    return players
    
def get_country(country_link, driver):
    driver.get(country_link)
    wait = WebDriverWait(driver,10)
    league = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="yw1"]/table/tbody/tr[2]/td[1]/table/tbody/tr/td[2]/a')))
    players = get_league(league.get_attribute('href'), driver)
    return players

def get_all():
    players = []
    driver = get_driver()
    driver.get('https://www.transfermarkt.com/wettbewerbe/europa')
    accept_cookies(driver)
    wait = WebDriverWait(driver,10)
    country_map = wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="europa_Map"]')))
    countries = country_map.find_elements_by_tag_name('area')
    country_dict = {country.get_attribute('title'): country.get_attribute('href') for country in countries}
    for key, value in country_dict.items():
        print(key)
        if key == 'Montenegro':
            players += get_league(value, driver)
        else:
            players += get_country(value, driver)
    driver.close()
    return players

In [190]:
players = get_all()
pd.DataFrame(players).to_csv("all.tsv", sep="\t", index=False)

Bosnia & Herzegovina
hsk-zrinjski-mostar
fk-sarajevo
fk-borac-banja-luka
fk-tuzla-city
fk-velez-mostar
fk-zeljeznicar-sarajevo
nk-siroki-brijeg
fk-sloboda-tuzla
hsk-posusje
fk-leotar-trebinje
fk-rudar-prijedor
fk-radnik-bijeljina
Croatia
gnk-dinamo-zagreb
hnk-hajduk-split
nk-osijek
hnk-rijeka
nk-lokomotiva-zagreb
hnk-gorica
hnk-sibenik
slaven-belupo-koprivnica
nk-hrvatski-dragovoljac
nk-istra-1961
Serbia
red-star-belgrade
fk-partizan-belgrade
fk-cukaricki
fk-vojvodina-novi-sad
fk-tsc-backa-topola
fk-spartak-subotica
fk-napredak-krusevac
fk-radnik-surdulica
fk-radnicki-nis
fk-metalac-gornji-milanovac
fk-vozdovac
fk-kolubara-lazarevac
fk-mladost-lucani
fk-proleter-novi-sad
fk-novi-pazar
fk-radnicki-1923-kragujevac
Slovenia
nk-olimpija-ljubljana
nk-maribor
nk-celje
ns-mura
fc-koper
nk-aluminij-kidricevo
nk-domzale
nk-bravo
nk-tabor-sezana
nk-radomlje
Hungary
ferencvarosi-tc
mol-fehervar-fc
puskas-akademia-fc
budapest-honved-fc
kisvarda-fc
ujpest-fc
debreceni-vsc
mtk-budapest
mez%C5%91kove

TimeoutException: Message: timeout: Timed out receiving message from renderer: 298.421
  (Session info: chrome=102.0.5005.115)


In [185]:
team_link = 'https://www.transfermarkt.com/fc-chelsea/kader/verein/631'
team_link.split('/')[3]


'fc-chelsea'

In [201]:
players = []
for key, value in list(country_dict.items())[-8:]:
    print(key)
    driver = get_driver()
    if key == 'Montenegro':
        players += get_league(value, driver)
    else:
        players += get_country(value, driver)
    driver.close()

Kazakhstan
fc-astana
tobol-kostanay
kairat-almaty
fk-aktobe
fk-aksu
fk-taraz
shakhter-karaganda
qyzyljar-petropavlovsk
kaspiy-aktau
fk-turan
ordabasy-shymkent
fk-atyrau
fk-maktaaral
akzhayik-uralsk
Macedonia
shkendija-tetovo
fc-shkupi
makedonija-gjorce-petrov
akademija-pandev
bregalnica-stip
renova-dzepciste
struga-trim-amp-lum
rabotnicki-skopje
borec-veles
tikves-kavadarci
fk-skopje
pelister-bitola
Kosovo
fc-llapi
kf-ballkani
fc-drita-gjilan
fc-prishtina
sc-gjilani
kf-drenica
fc-malisheva
kf-ulpiana
kf-dukagjini
kf-feronikeli-drenas
Montenegro
buducnost-podgorica
fk-sutjeska-niksic
fk-decic-tuzi
fk-iskra-danilovgrad
ofk-petrovac
rudar-pljevlja
fk-jezero
fk-mornar-bar
fk-podgorica
fk-zeta-golubovac
Gibraltar
lincoln-red-imps-fc
fc-brunos-magpies
st-josephs-fc
europa-fc
glacis-united
manchester-62-fc
fc-college-1975
mons-calpe-sc
lynx-fc
lions-gibraltar-fc
europa-point-fc
Liechtenstein
San Marino
la-fiorita-1967
sp-tre-penne
ac-libertas
tre-fiori-fc
ss-folgore-falciano
ss-virtus
ss-penn

In [196]:
list(country_dict).index('Russia')

23

In [202]:
def map_club_to_country(club_list):
    driver = get_driver()
    driver.get('https://www.transfermarkt.com/')
    accept_cookies(driver)
    club_country = {}
    for club in club_list:
        search = driver.find_elements_by_xpath('//*[@id="schnellsuche"]/input')
        search[0].clear()
        search[0].send_keys(club)
        search[1].click()
        country = driver.find_element_by_xpath('//*[@id="yw0"]/table/tbody/tr[1]/td[3]/img').get_attribute('title')
        club_country[club] = country
    return club_country

In [203]:
driver = get_driver()
driver.get('https://www.transfermarkt.com/')
accept_cookies(driver)

In [204]:
search = driver.find_elements_by_xpath('//*[@id="schnellsuche"]/input')
search[0].send_keys('Paris Saint-Germain')
search[1].click()

In [206]:
search = driver.find_elements_by_xpath('//*[@id="schnellsuche"]/input')
search[0].clear()
search[0].send_keys('Chelsea FC')
search[1].click()

In [207]:
country = driver.find_element_by_xpath('//*[@id="yw0"]/table/tbody/tr[1]/td[3]/img').get_attribute('title')
country

'England'

In [213]:
def get_first_league_clubs2020():
    club_country = {}
    country_ranking = {}
    driver = get_driver()
    driver.get('https://www.transfermarkt.com/wettbewerbe/europa')
    accept_cookies(driver)
    wait = WebDriverWait(driver,10)
    country_map = wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="europa_Map"]')))
    countries = country_map.find_elements_by_tag_name('area')
    country_dict = {country.get_attribute('title'): country.get_attribute('href') for country in countries}
    for key, value in list(country_dict.items()):
        print(key)
        if key != 'Montenegro':
            driver.get(value)
            league = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="yw1"]/table/tbody/tr[2]/td[1]/table/tbody/tr/td[2]/a')))
            league_link = league.get_attribute('href')
        else:
            league_link = value
        driver.get(league_link + '/plus/?saison_id=2020')
        teams = driver.find_elements_by_xpath('//*[@id="yw1"]/table/tbody/tr')
        team_names = [team.find_elements_by_tag_name('td')[1].find_elements_by_tag_name('a')[0].text for team in teams]
        ranking = driver.find_elements_by_xpath('//*[@id="wettbewerb_head"]/div/div/div[2]/div[2]/div[2]/table/tbody/tr[1]/td/a').text
        for name in team_names:
            club_country[name] = key
        country_ranking[key] = ranking
    return [club_country, country_ranking]

In [ ]:
club_country, country_ranking = get_first_league_clubs2020()
club_country

In [ ]:
result_df = pd.DataFrame(club_country.items())
result_df.to_csv('clubs.tsv', sep='\t', index=False, header=False)